<img class="img-responsive" width="50%" src="./source_code/logo.jpeg" alt="logo">

In [1]:
import sys
import os
import json
import pandas as pd
import numpy as np

In [2]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as ipw
from IPython.core.display import HTML, display
from IPython.display import clear_output

In [3]:
# Warning: This code changes the layout of the ENTIRE notebook (specifically, removes the margins)
# to reverse this feature you must rerun this cell with different numbers
display(HTML("<style> *{margin:0; padding:0;} html, body, \
             .container{margin:0;!important padding:0;!important} \
             .container { width:100% !important;}</style>"))

# Welcome to Judoscape!

<font color="magenta">**Judoscape**</font> is a [jupyter](http://jupyter.org/index.html) based framework for for easily running cytoscape with ** *no coding knowledge required* **. The entire platform is cloud based so you don't need to download anything, just login and start. Furthermore, sharing scientific results is a breeze. When you're done with this notebook, give it to your colleagues and they can instantly replicate it on <font color="magenta">Judoscape</font>.

<font color="red">**Before you continue**</font> please watch the tutorial video below to get a better idea of what the <font color="magenta">Judoscape</font> software is and how it works.

In [4]:
HTML("""
<div class="row"><div class="col-xs-12 col-md-offset-3 col-md-6">
<div class="embed-responsive embed-responsive-16by9">
  <iframe class="embed-responsive-item" 
  src='https://www.youtube.com/embed/tgbNymZ7vqY'></iframe></div>
</div></div>
""")

# Build a Dataset

In this section will will choose a dataset to work on. There are **2** ways to create a datast:

1. Choose a dataset on this machine from [my_data](./my_data)
2. Choose a dataset from the internet

<font color="magenta">**From my_data : **</font>The folder, [my_data](./my_data), contains datasets that you can load into this notebook. You can also upload a dataset from your computer to the [my_data](./my_data) folder. To view new files added to [my_data](./my_data) update the program by running all the cells again.


<font color="magenta">** The Internet : **</font> A dataset can be downloaded from the internet as well. Type the <font color="green">URL</font> of the dataset to download it into this notebook.

In [5]:
# We create the First widget for selecting Data
widg1_text = ipw.HTML("<h5>Select a dataset from the folder <a href='./my_data'>my_data</a> :</h5>")
files = os.listdir("./my_data")
files = [ x for x in files if ".sif" in x]
widg1_select = ipw.Dropdown(options=files, description='Files :')
widg1 = ipw.VBox([widg1_text, widg1_select])

In [6]:
# We create the Second widget for Downloading from a website
widg2_text = ipw.HTML("<h4 style='color: red'>Note: This functionality is not complete yet</h4> \
                        <h5>Type the URL of the dataset to download :</h5>")
widg2_select = ipw.Text(options=files, description='Data URL :')
widg2 = ipw.VBox([widg2_text, widg2_select])

In [7]:
# We put both widgets into a tab widget and display it
tab = ipw.Tab()
tab.children = [widg1, widg2]
tab.set_title(0, "my_data Folder")
tab.set_title(1, "The Internet")
tab

A Jupyter Widget

In [8]:
# We now get the data's filename according to the user's selection from the tab widget
data_btn= ipw.Button(description='Get Data',button_style='info', tooltip="Use the above box to import data" )
display(data_btn)

df, nodes = None, None  
def on_get_data_clicked(b):
    global df; global nodes;  #<-- this is where we put the dataset's dataframe and the node names
    clear_output();display(data_btn)  #<-- in case the button was clicked before, we reset output
    
    if tab.selected_index == 1:  #<--  What to do if data comes from internet
        display(HTML("<h4 style='color: red'>This functionality is not complete. Use my_data folder</h4>"))
        return
    elif( tab.selected_index == 0 ): #<--  What to do if data comes from local folder
        filename = widg1_select.value
        display(HTML("<hr><font size=4><b style='color: magenta'>Using Dataset : </b> "+filename+"</font>"))
        df = pd.read_csv("./my_data/"+filename, delim_whitespace=True, header=None) #<-- reading in data
        df= pd.DataFrame( df[ (df[0]!='?')&(df[1]!='?')&(df[2]!='?') ])  #<-- remove missing data values
        nodes = np.unique( df[0].values.tolist() + df[2].values.tolist() )  #<-- get nodes
    
data_btn.on_click(on_get_data_clicked)

A Jupyter Widget

# Visualize the Data

## Data Summary

Now that you have chosen a dataset, lets look at some basic data features to get a better understanding of what is happening. **Click the button** below to see how many <font color="blue">nodes</font> and  <font color="blue">edges</font> your data contains, and to view the first few datapoints.

In [9]:
# We now get the data's filename according to the user's selection from the tab widget
summary_btn= ipw.Button(description='View Summary',button_style='info', tooltip="Click for Data Summary" )
display(summary_btn)

def on_summary_clicked(b):
    clear_output(); display(summary_btn)  #<-- in case the button was clicked before, we reset output
    if type(df) == type(None):   #<-- this occures if user didn't click the "Get Data" button 
        display(HTML("<h3 style='color: red'>You must First Build a Dataset</h3>"))
    else :
        display(HTML("<hr><p><b>Number of Nodes : </b>"+str(len(nodes))+"</p> \
                      <p><b>Number of Edges : </b>"+str(len(df))+"</p>"))
        display(df[0:7])
summary_btn.on_click(on_summary_clicked)

A Jupyter Widget

,0,1,2
0,YKR026C,pp,YGL122C
1,YGR218W,pp,YGL097W
2,YGL097W,pp,YOR204W
3,YLR249W,pp,YPR080W
4,YLR249W,pp,YBR118W
5,YLR293C,pp,YGL097W
6,YMR146C,pp,YDR429C


## Create Graph

We will now visualize our dataset using a graph. You can use the tools below to change the <font color="blue">layout</font> of where nodes are placed as well as to change the graph's <font color="blue">color</font>. The  <font color="blue">layout</font> tool is especially useful for examining different characteristics in your data. After you decide the graph's settings, click the button below to **create your graph**.

In [10]:
# We save our graph into an html file and call the html file to create the graph
# This is the basic template for the html, we then add the javascript code of the graph to it
html = """<html>
    <head><script src="http://blog.js.cytoscape.org/public/demos/getting-started/cytoscape.js"></script></head>
    <style> #cy { width: 100%; height: 100%; position: absolute; top: 0px; left: 0px;} </style>
    <body><div id="cy"></div></body>
</html>\n\n"""

In [11]:
# Helper Functions: getLayout and getStyle

def getLayout(layout): #<-- creates the layout when we create the graph
    return "layout: { name: '"+layout+"' }, \n"

def getStyle( color ): #<-- uses widg_color to create the style (specifically the color) of the graph
    return "style: [ { selector: 'node', style: { 'height': 20, 'width': 20, 'background-color': '"+color+"' } },\n \
{ selector: 'edge', style: { 'width': 5, 'opacity': 0.5, 'line-color': '"+color+"'} } ], \n "

In [12]:
# We create the widgets for graph settings
widg_layout = ipw.Dropdown(options=['cose','circle','grid','random','breadthfirst'], description='Layout : ')
widg_color = ipw.ColorPicker(  description='Color : ', value='gray' )
display( ipw.HBox([widg_layout ,widg_color]) )

A Jupyter Widget

In [13]:
js_head = "<script> var cy = cytoscape({ container: document.getElementById('cy'), \n" #<-- starting code
js_tail="\n }); </script>" #<-- ending code

In [14]:
### Helper Function: getElements(df)
# This create the nodes/edges in the correct format for cytoscape.js
def getElements(df):
    # Add nodes to the js file
    js_elements = "elements: [ \n"
    for i in nodes:
        js_elements += "{ data: { id: '"+i+"'} }, \n"
        
    # Add edges to the js file
    for i in range(len(df)):
        word = df.iloc[i].values
        js_elements += "{data: { id:'"+str(i)+"', source:'"+ str(word[0])+"', target:'"+str(word[2])+"' } }, \n"    
    js_elements += "]\n"
    
    return js_elements

In [15]:
# We now get the data's filename according to the user's selection from the tab widget
graph_btn= ipw.Button(description='Create Graph!',button_style='info', tooltip="Click to Create Graph!" )
display(graph_btn)

def on_graph_clicked(b):
    clear_output(); display(graph_btn)  #<-- in case the button was clicked before, we reset output
    if type(df) == type(None):   #<-- this occures if user didn't click the "Get Data" button 
        display(HTML("<h3 style='color: red'>You must First Build a Dataset</h3>"))
    # This creates the graph!
    else : 
        # Get js templates with corresponding helper functions
        js_layout = getLayout( widg_layout.value )
        js_style  = getStyle( widg_color.value )
        js_elements = getElements(df)
           
        js = js_head+ js_layout+ js_style+ js_elements+ js_tail #<-- create final js file of graph
        with open("./source_code/graph.html","w") as file: #<-- Add html and js to the HTML file
            file.write( html+js )
        display(HTML("""<hr>
        <div class="embed-responsive embed-responsive-4by3">
          <iframe class="embed-responsive-item" 
          src='./source_code/graph.html'></iframe>
        </div><hr>
        """)) #<-- display the newly created html file

graph_btn.on_click(on_graph_clicked)

A Jupyter Widget

In [16]:
# This just adds a little space to the bottom of the notebook
display(HTML("<p>&nbsp</p><p>&nbsp</p><p>&nbsp</p><p>&nbsp</p><p>&nbsp</p>"))